In [38]:
!pip install -q git+https://github.com/nikitakapitan/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [1]:
import torch
import torch.nn as nn
from transformers.main import make_model

from transformers.Embeddings import Embeddings
from transformers.PositionalEncoding import PositionalEncoding
# from transformers.A import A
# from transformers.A import A
# from transformers.A import A

ModuleNotFoundError: No module named 'transformers'

In [40]:
D_MODEL = 16
D_FF = 2048
H = 8
DROPOUT = 0.1
N = 2


FRENCH = 11 #SOURCE : number of all words in French  vocab
ENGLISH  = 11 #TARGET : number of all words in English vocab

# INITIALIZE
test_model = make_model(src_vocab=FRENCH, tgt_vocab=ENGLISH, N=N)
test_model.eval() # switch to eval (no DropOut & BatchNorm learning)

# INFERENCE
src = torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]])
src_mask = torch.ones(1,1,10)

# memory = test_model.encode(src, src_mask)


# Down-brake **memory = test_model.encode(src, src_mask)**

## Embeddings(vocab, d_model)

In [72]:
x = src
x.shape

torch.Size([1, 10])

In [73]:
emb = Embeddings(vocab=FRENCH, d_model=D_MODEL)
x = emb(x)
x.shape

torch.Size([1, 10, 16])

## PositionalEncoding(d_model, dropout, max_len)

In [74]:
MAX_LEN = 5000
import math
pos_enc = PositionalEncoding(d_model=D_MODEL, dropout=DROPOUT, max_len=MAX_LEN)
x = pos_enc(x)
x.shape

RuntimeError: ignored

In [81]:
# debug
max_len = MAX_LEN
d_model = D_MODEL

pe = torch.zeros(max_len, d_model)
print(pe.shape)
position = torch.arange(0, max_len).unsqueeze(1) # (max_len, 1)
div_term = torch.exp(
    torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
)
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
pe = pe.unsqueeze(0) # (1, max_len, d_model)
pe.shape

torch.Size([5000, 16])


torch.Size([1, 5000, 16])

In [57]:
pe.shape

torch.Size([5000, 16])

In [60]:
pe=pe.unsqueeze(0)
pe.shape

torch.Size([1, 5000, 16])

In [84]:
print(x.shape)
x.size(1)

torch.Size([1, 10, 16])


10

In [85]:
pe[:,x.size(1), :].shape

torch.Size([1, 16])